In [2]:
int(sqrt(313))

17

In [5]:
import gmpy2
from Crypto.Util.number import *

p = getPrime(313)
q = getPrime(313)

r = 14

el, eu = gmpy2.mpfr(0.313370), gmpy2.mpfr(0.313371)
n = p ** r * q
LB, UB = gmpy2.mpfr(n ** el), gmpy2.mpfr(n ^ eu)

LB, UB

(mpfr('2.2742059900721544e+442'), mpfr('2.2816099967933556e+442'))

In [6]:
d = getRandomRange(int(LB), int(UB))

In [7]:
int(d).bit_length()

1470

In [8]:
n.bit_length()

4690

In [10]:
int(4690 / 1470)

3

In [12]:
from math import log
r = 14
(log(1 + r)/r) * 313

60.544265210356556

In [14]:
int(313/(r+1))

20

In [15]:
import itertools

def small_roots(f, bounds, m=1, d=None):
	if not d:
		d = f.degree()

	if isinstance(f, Polynomial):
		x, = polygens(f.base_ring(), f.variable_name(), 1)
		f = f(x)

	R = f.base_ring()
	N = R.cardinality()
	
	f /= f.coefficients().pop(0)
	f = f.change_ring(ZZ)

	G = Sequence([], f.parent())
	for i in range(m+1):
		base = N^(m-i) * f^i
		for shifts in itertools.product(range(d), repeat=f.nvariables()):
			g = base * prod(map(power, f.variables(), shifts))
			G.append(g)

	B, monomials = G.coefficient_matrix()
	monomials = vector(monomials)

	factors = [monomial(*bounds) for monomial in monomials]
	for i, factor in enumerate(factors):
		B.rescale_col(i, factor)

	B = B.dense_matrix().LLL()

	B = B.change_ring(QQ)
	for i, factor in enumerate(factors):
		B.rescale_col(i, 1/factor)

	H = Sequence([], f.parent().change_ring(QQ))
	for h in filter(None, B*monomials):
		H.append(h)
		I = H.ideal()
		if I.dimension() == -1:
			H.pop()
		elif I.dimension() == 0:
			roots = []
			for root in I.variety(ring=ZZ):
				root = tuple(R(root[var]) for var in f.variables())
				roots.append(root)
			return roots

	return []

In [16]:
bits = 1024
errs = 16
locs = sorted(Subsets(range(bits), errs).random_element())
locs

[205,
 308,
 350,
 401,
 408,
 450,
 475,
 593,
 607,
 686,
 785,
 786,
 837,
 887,
 911,
 944]

In [23]:
_b = '1'
b = '101001001'

nbit = len(b)

o1 = [b[:i] or "L" for i in range(nbit) if b[i] == _b]
print(o1)

_b = '0'
b = '010111111'

nbit = len(b)

o1 = [b[:i] or "L" for i in range(nbit) if b[i] == _b]
print(o1)

['L', '10', '10100', '10100100']
['L', '01']


In [22]:
256*0.666

170.496000000000

In [139]:
key = os.urandom(32)
key.hex()

'edf15612efa0ddc4f3df41ec3fe325986b49eeac5be7781f158572afa9f1acdf'

In [141]:
import socket

def sendline(sock, data):
    sock.sendall(data + b"\n")

def recvline(sock):
    buf = b""
    while True:
        data = sock.recv(1)
        if not data:
            break
        buf += data
        if data == b"\n":
            break
    return buf

def recvuntil(sock, delim):
    buf = b""
    while True:
        data = sock.recv(1)
        if not data:
            break
        buf += data
        if delim in buf:
            break
    return buf


In [140]:
# First define the field
R.<x> = PolynomialRing(GF(2))
# Convert f to binary polynomial
f_poly = R([int(b) for b in bin(0x1f3267f571be716d65f11ecb21b86d2e9)[2:][::-1]])
#print(f_poly)
# Create the quotient field
F.<a> = GF(2^128, modulus=f_poly)

def to_field_element(byte_string):
    """Convert bytes to field element"""
    n = int.from_bytes(byte_string, byteorder = 'big')
    f = R([int(b) for b in bin(n)[2:][::-1]])
    #print(f)
    f = F(f)
    #print(f)
    return f#F.fetch_int(n)

def field_element_to_int(elem):
    """Convert field element back to integer"""
    # Get the polynomial representation
    poly = elem.polynomial()
    # Convert polynomial coefficients to integer
    result = 0
    for i, coeff in enumerate(poly.coefficients(sparse=False)):
        if coeff == 1:
            result |= (1 << i)
    return result

def solve_for_key(combined_elem, msg):
    """
    Given the field element for (key || len(msg) || msg) and the msg,
    solve for the key
    """
    # Known part: len(msg) || msg
    known_part = bytes([len(msg)]) + msg
    known_int = int.from_bytes(known_part, byteorder = 'big')
    known_elem = F(R([int(b) for b in bin(known_int)[2:][::-1]]))
    
    # Calculate the power of a we need
    shift = (1 + len(msg)) * 8
    
    # In the field: key_elem = (combined_elem - known_elem) * (a^shift)^(-1)
    shifter = F(a)^shift
    shifter_inv = shifter^(-1)  # Modular multiplicative inverse in the field
    
    key_elem = F(combined_elem - known_elem) * F(shifter_inv)

    return key_elem

def mac_poly(key_bytes, msg):
    """Implement the MAC operation in the field"""
    # Combine key, length, and message as in the original
    print(to_field_element(key_bytes))
    combined = key_bytes + bytes([len(msg)]) + msg
    # Convert to field element
    elem = to_field_element(combined)
    #print(elem)
    r = solve_for_key(elem, msg)
    print(r)
    # Compute the 10th power in the field
    return elem, elem^10

# Example usage:
#key = os.urandom(32)
msg = b"a"
og, tag = mac_poly(key, msg)
#print(tag)
# root = tag.nth_root(10000, all = True)
# for r in root:
#     print(r == og)
#print(tag)
tag = field_element_to_int(tag)
print(tag)

a^127 + a^126 + a^125 + a^124 + a^123 + a^122 + a^121 + a^120 + a^118 + a^117 + a^116 + a^114 + a^112 + a^110 + a^109 + a^108 + a^106 + a^104 + a^103 + a^101 + a^98 + a^97 + a^96 + a^95 + a^93 + a^91 + a^89 + a^88 + a^85 + a^84 + a^79 + a^77 + a^76 + a^75 + a^73 + a^71 + a^68 + a^67 + a^66 + a^65 + a^63 + a^61 + a^60 + a^58 + a^57 + a^51 + a^50 + a^49 + a^48 + a^47 + a^44 + a^43 + a^41 + a^39 + a^38 + a^37 + a^36 + a^35 + a^34 + a^32 + a^31 + a^30 + a^27 + a^25 + a^24 + a^21 + a^20 + a^18 + a^15 + a^14 + a^9 + a^8 + a^7 + a^5 + a^4 + a^2 + a + 1
a^127 + a^126 + a^125 + a^124 + a^123 + a^122 + a^121 + a^120 + a^118 + a^117 + a^116 + a^114 + a^112 + a^110 + a^109 + a^108 + a^106 + a^104 + a^103 + a^101 + a^98 + a^97 + a^96 + a^95 + a^93 + a^91 + a^89 + a^88 + a^85 + a^84 + a^79 + a^77 + a^76 + a^75 + a^73 + a^71 + a^68 + a^67 + a^66 + a^65 + a^63 + a^61 + a^60 + a^58 + a^57 + a^51 + a^50 + a^49 + a^48 + a^47 + a^44 + a^43 + a^41 + a^39 + a^38 + a^37 + a^36 + a^35 + a^34 + a^32 + a^31 + a

In [154]:
from Crypto.Util.number import long_to_bytes

def conn():
  sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
  sock.connect(('188.245.36.243', 7700))
  return sock

def get(io, c):
    to_send = b'query ' + c.encode().hex().encode()
    sendline(io, to_send)
    r = bytes.fromhex(recvline(io).decode().strip())
    return to_field_element(r)

def go(io):
    roots = []
    for c in ['a', 'b', 'c', 'd', 'e']:
        f = get(io, c)
        st = []
        for root in f.nth_root(1000000, all=True): #for part 1 change the root value to 10
            f = solve_for_key(root, c.encode())
            st.append(f)
        roots.append(st)
    #print(roots)
    key = None
    for root in roots[0]:
        ye = True
        for r in roots[1:]:
            if root not in r:
                ye = False
                break
        if ye:
            print('key found')
            print(root)
            key = root
            break
            
    if key is None:
        print('Need to rerun')
        return False

    key = field_element_to_int(key)
    sendline(io, b'solve ' + long_to_bytes(int(key)).hex().encode())
    
    r = recvline(io).decode().strip()
    print(r)
    if 'wrong key' in r:
        print('Need rerun??')
        return False
    return True


for _ in range(100):
    r = go(conn())
    if r:
        break
    
#hxp{1F_y0U_C4n_r34D_tH1s_I_gU3sS_i7_w4s_n0T_s4F3_4fT3r_4lL} 
#hxp{L1f3_1s_4_CiRCuS_4nD_w3_4rE_7he_cL0wNz}

key found
a^126 + a^125 + a^124 + a^122 + a^121 + a^120 + a^115 + a^114 + a^113 + a^112 + a^110 + a^104 + a^103 + a^102 + a^100 + a^99 + a^95 + a^94 + a^93 + a^91 + a^90 + a^88 + a^86 + a^85 + a^83 + a^82 + a^81 + a^80 + a^78 + a^77 + a^73 + a^71 + a^69 + a^68 + a^61 + a^57 + a^52 + a^48 + a^46 + a^44 + a^43 + a^40 + a^39 + a^35 + a^31 + a^30 + a^28 + a^27 + a^25 + a^23 + a^22 + a^18 + a^17 + a^7 + a^4 + a^3 + a + 1
hxp{L1f3_1s_4_CiRCuS_4nD_w3_4rE_7he_cL0wNz}


In [66]:
import os

# First define the field
R.<x> = PolynomialRing(GF(2))
# Convert f to binary polynomial
f_poly = R([int(b) for b in bin(0x1f3267f571be716d65f11ecb21b86d2e9)[2:]])
# Create the quotient field
F.<a> = GF(2^128, modulus=f_poly)

def to_field_element(byte_string):
    """Convert bytes to field element"""
    # Use little-endian to match original implementation
    n = int.from_bytes(byte_string, byteorder='little')
    f = R([int(b) for b in bin(n)[2:]])
    return F(f)

def field_element_to_int(elem):
    """Convert field element back to integer"""
    poly = elem.polynomial()
    result = 0
    for i, coeff in enumerate(poly.coefficients(sparse=False)):
        if coeff == 1:
            result |= (1 << i)
    return result

def solve_for_key(combined_elem, msg):
    """
    Given the field element for (key || len(msg) || msg) and the msg,
    solve for the key
    """
    # Known part: len(msg) || msg
    known_part = bytes([len(msg)]) + msg
    known_elem = to_field_element(known_part)
    
    # Calculate the power of a we need
    shift = (1 + len(msg)) * 8
    
    # In the field: key_elem = (combined_elem - known_elem) * (a^shift)^(-1)
    shifter = a^shift
    shifter_inv = shifter^(-1)
    
    key_elem = (combined_elem - known_elem) * shifter_inv
    return key_elem

def mac_poly(key_bytes, msg):
    """Implement the MAC operation in the field"""
    # Combine key, length, and message
    combined = key_bytes + bytes([len(msg)]) + msg
    # Convert to field element
    elem = to_field_element(combined)
    
    # Get original key for verification
    recovered_key_elem = solve_for_key(elem, msg)
    print("Original key element:", to_field_element(key_bytes))
    print("Recovered key element:", recovered_key_elem)
    
    # Compute the 10th power in the field
    return elem, elem^10

# Test
key = os.urandom(32)
msg = b"test message"
og, tag = mac_poly(key, msg)

# Verify that we can recover the original element
root = tag^((F.order() - 1) // 10 * 9)  # Calculate 10th root
print("\nRoot matches original:", root == og)

Original key element: a^127 + a^125 + a^124 + a^123 + a^122 + a^121 + a^120 + a^118 + a^116 + a^115 + a^114 + a^112 + a^110 + a^109 + a^104 + a^102 + a^101 + a^99 + a^95 + a^93 + a^90 + a^89 + a^88 + a^82 + a^81 + a^80 + a^79 + a^76 + a^73 + a^72 + a^71 + a^69 + a^68 + a^64 + a^62 + a^60 + a^59 + a^58 + a^57 + a^54 + a^53 + a^47 + a^45 + a^41 + a^40 + a^39 + a^35 + a^32 + a^31 + a^30 + a^29 + a^28 + a^25 + a^24 + a^22 + a^20 + a^19 + a^18 + a^17 + a^16 + a^12 + a^9 + a^8 + a^7 + a^3 + a + 1
Recovered key element: a^127 + a^125 + a^118 + a^116 + a^115 + a^105 + a^103 + a^102 + a^97 + a^96 + a^95 + a^93 + a^91 + a^87 + a^86 + a^78 + a^77 + a^75 + a^74 + a^72 + a^69 + a^67 + a^66 + a^63 + a^62 + a^60 + a^58 + a^53 + a^48 + a^44 + a^40 + a^38 + a^34 + a^30 + a^29 + a^26 + a^25 + a^23 + a^18 + a^16 + a^15 + a^14 + a^12 + a^11 + a^10 + a^8 + a^4 + a

Root matches original: False


In [108]:
#!/usr/bin/env python3

f = 0x1f3267f571be716d65f11ecb21b86d2e9

def to_bits(bs):
    return int.from_bytes(bs)

def from_bits(v):
    return int.to_bytes(v, 16)

def red(x):
    while (l := x.bit_length()) > 128:
        x ^= f << l - 129
    return x

def mul(x, y):
    z = 0
    for i in range(x.bit_length()):
        if (x >> i) & 1:
            z ^= y << i
    return red(z)

def exp(x, n):
    assert n >= 0
    if not n:
        return 1
    if n % 2:
        return mul(x, exp(x, n-1))
    return exp(mul(x, x), n//2)

class Mac:
    def __init__(self, key):
        self.key = key

    def __call__(self, msg):
        tag = exp(to_bits(self.key + bytes([len(msg)]) + msg), 10)
        return from_bits(tag)

mac = Mac(key)
print(mac(b'a'))


TypeError: from_bytes() missing required argument 'byteorder' (pos 2)

In [160]:
p, q = getPrime(512), getPrime(512)
phi = (p - 1)*(q-1)
e = 0x10001
d = pow(e, -1, phi)
n = int(p*q)
d = int(d)
n.bit_length(), d.bit_length()

(1024, 1020)

In [165]:
bits = 1024
errs = 16
for _ in range(100):
    locs = sorted(Subsets(range(bits), errs).random_element())
    s = locs[:8]
    print(s)

[93, 112, 118, 210, 218, 228, 246, 283]
[89, 103, 138, 195, 246, 260, 345, 374]
[28, 67, 125, 147, 184, 309, 318, 319]
[53, 60, 85, 266, 279, 284, 310, 413]
[51, 83, 108, 127, 161, 178, 226, 306]
[2, 16, 68, 95, 248, 257, 280, 313]
[121, 203, 234, 327, 372, 429, 434, 445]
[21, 123, 137, 167, 234, 263, 516, 542]
[54, 82, 91, 104, 174, 279, 307, 392]
[78, 97, 106, 143, 258, 446, 561, 584]
[51, 156, 159, 245, 310, 342, 410, 480]
[6, 157, 212, 332, 342, 367, 415, 484]
[55, 119, 158, 169, 190, 345, 353, 459]
[109, 189, 263, 341, 408, 463, 469, 470]
[21, 67, 392, 411, 495, 496, 544, 555]
[52, 104, 125, 160, 221, 234, 319, 336]
[17, 27, 207, 340, 410, 422, 704, 712]
[67, 107, 237, 256, 305, 384, 495, 513]
[13, 29, 77, 119, 171, 209, 306, 321]
[11, 32, 170, 214, 314, 371, 437, 507]
[52, 147, 150, 211, 253, 323, 470, 541]
[37, 136, 137, 145, 151, 213, 331, 346]
[21, 76, 143, 241, 278, 320, 333, 428]
[53, 175, 211, 255, 397, 420, 590, 640]
[87, 91, 207, 275, 283, 343, 397, 560]
[2, 160, 215, 233

In [166]:
s = '10110000011000 10100011101000 10101010000000 100000101011000 10010100110000 1101000001000 11001110001000 11011010111000 11001001111000 10010100110000 10100111111000 10101110010000 11001001111000 1001010011000 10100001100000 1100110000000 11111010110000 11010110101000 100001001101000'

for c in s.split():
    c = int(c, 2)
    c = c // (69+67)
    print(chr(c), end='')

SMP{F1ag_FOR_#L0ve}

In [ ]:
P1: 7307039331
P2_O: 6983432688
P3: 5284255017203633261370535345682263310336
K: 9531
Y: 11709967940066793762796184140152610769821716917529318252119959861290032840327
H: 730699457350798390839309068289663772131566530906536306094973


Hint: Do an Axe-Ore between 'usedistofindouttheseed' and 'thisisthekeytogetyourseed' to get the numerical value you need. 
Alice loved splitting and scrambling numbers, but every piece has its place... eventually.

In [168]:
y = 11709967940066793762796184140152610769821716917529318252119959861290032840327
y.bit_length()

253

In [235]:
s1= 7307039331
s2= 6983432688
s3= 5284255017203633261370535345682263310336
s4= 9531
s5 = s2^^s4
s5

6983425227

In [199]:
h = 730699457350798390839309068289663772131566530906536306094973
#h *= s3
h.bit_length()

199

In [192]:
c1 = b'usedistofindouttheseed'
c2 = b'thisisthekeytogetyourseed'
print(len(c1), len(c2))
c1 = bytes_to_long(c1)
c2 = bytes_to_long(c2)
x = c1 ^^ c2
print(x)
for s in [s1, s2, s3, s4]:
    print(s, str(s) in str(x))

22 25
730703933169834252275284255017203633261370535345682263310336
7307039331 True
6983432688 False
5284255017203633261370535345682263310336 True
9531 False


In [196]:
l1=6983432688
l2=6983425227
l1-l2

7461

In [251]:
def xor(sa, sb):
    return bytes([a^^b for a, b in zip(sa, sb)])

c1 = b'usedistofindouttheseed'
c2 = b'thisisthekeytogetyourseed'
c1 = b'\x00'*3 + c1
#c1 = c1 + b'\x00'*3
assert len(c1) == len(c2)
x = xor(c1, c2)
x = bytes_to_long(x)
x

730703933169834252275284255017203633261370535345682263310336

In [205]:


import random
random.seed(x)

p = getrandbits(256)
while True:
    if is_prime(p):
        print(p)
        break
    p += 1

39398718916463592198674075927356670680922454483198614088152949416703437115373


In [238]:
from itertools import permutations

def all_concatenations_as_ints(nums):
    """
    Generate all possible concatenations of the integer strings 
    in their integer form.

    Args:
        nums (list): List of integers.

    Returns:
        list: List of integers representing all concatenations.
    """
    # Convert each integer to a string
    str_nums = list(map(str, nums))
    
    # Generate all permutations of the string list
    permuted_strings = permutations(str_nums)
    
    # Concatenate the permutations and convert back to integers
    concatenated_ints = [int("".join(p)) for p in permuted_strings]
    
    return concatenated_ints

# Example usage
nums = [s1, s5, s3, s4]
result = all_concatenations_as_ints(nums)
print(result[0].bit_length())
tt = s1*s2*s3*s4*h
print(tt.bit_length())


213
410


In [239]:
def dec(q, p, e, c):
    assert is_prime(p) and is_prime(q)
    phi = (p - 1) * ( q- 1)
    d = pow(e, -1, phi)
    m = pow(c, d, p * q)
    f = long_to_bytes(ZZ(m))
    if b'SMP' in f:
        print(f)

p = 39398718916463592198674075927356670680922454483198614088152949416703437115373
for re in result:
    dec(y, p, 0x10001, re)

print('END')

END


In [252]:
p = 39398718916463592198674075927356670680922454483198614088152949416703437115373
long_to_bytes(x ^^ h)

b'SMP{Numb3r_1S#_awes0m3}'

In [248]:
s = str(s1) + str(s2) + str(s3) + str(s4)
s = s1*s2*s3*s4
s.bit_length()

211

In [ ]:
P1: 4583053465
P2_O: 4712876638
P3: 050830727770097899766746520799989608903411206
K: 5964
Magic Number: 845321
Hidden Value: 45830534647166131662210698039087274510333927906501911560900149115


Hint:

"Axe-Ore the strings 'kshackzoneishereforyou' and 'ohiguessyouarerightwiththis'. 
Alice said it would give a 'seed'—but she wasn’t clear what kind of seed. 
Perhaps something was left behind to guide you? Or is it just another riddle?"

In [253]:
c1 = b'kshackzoneishereforyou'
c2 = b'ohiguessyouarerightwiththis'

c1 = b'\x00'*5 + c1
#c1 = c1 + b'\x00'*3
assert len(c1) == len(c2)
x = xor(c1, c2)
x = bytes_to_long(x)
x

45830534654712881426050830727770097899766746520799989608903411206

In [268]:
long_to_bytes(h ^^ x)

b'SMP{Y0u_g0t_x00r_k1tty}'

In [257]:


import random
random.seed(x)

p = getrandbits(256)
while True:
    if is_prime(p):
        print(p)
        break
    p += 1

5776552821453195782437569959830414871778558461341559077107341627942024484901


In [259]:
def dec(p, e, c):
    #assert is_prime(p) and is_prime(q)
    #phi = (p - 1) * ( q- 1)
    phi = p - 1
    d = pow(e, -1, phi)
    m = pow(c, d, p)
    f = long_to_bytes(ZZ(m))
    print(f)

#p = 39398718916463592198674075927356670680922454483198614088152949416703437115373
p = 5776552821453195782437569959830414871778558461341559077107341627942024484901
#for re in result:
dec(p, 845321, h)

print('END')

b'\x08]\x86\xf1\xb4\xd7T5BKvpl\xdd*\xf2\xd9\x19\xbcWt\x15\x98\xf1\xed\xd6pw\xfe4t\x02'
END


In [254]:
4712881426 ^^ 5964

4712876638

In [255]:
factor(845321)

29 * 103 * 283

In [256]:
h = 45830534647166131662210698039087274510333927906501911560900149115
h.bit_length()

215